# Калибровка друзей по интересам в группах Вконтакте. 

In [10]:
import collections
import vk
import pandas as pd
import time

Подключение VK API к проекту:

In [11]:
VK_TOKEN = '311cf83a74eb63d1a515fb243ce91ea95ebc73982ae77f0452c2338390042137481f6ece102db4c1aa400'
ID = 10385605
session = vk.Session(VK_TOKEN)
vk_api = vk.API(session, v=5.103)
deq = collections.deque(maxlen=4)

Определим методы для дальнейшей работы:

Метод возвращает список тематик групп по id пользователя.

In [3]:
def get_groups_activity(id):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    groups = vk_api.groups.get(user_id=id, fields=['activity'], extended=1)
    activities = []
    for group in groups['items']:
        try:
            activities.append(group['activity'])
        except Exception:
            continue
    return activities

Получаем процентное соотношение конкретного типа группы ко всем группам. Метод возвращает первые 10 по полуряности тематики.

In [4]:
def get_percents(activities):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    df = pd.DataFrame(data= activities, columns=['activity'])
    df = df.loc[(df.loc[:, 'activity'] != 'Открытая группа') & (df.loc[:, 'activity'] != 'Закрытая группа')]
    df['count'] = 1
    df2 = df.groupby('activity').count()
    df2['percent'] = df2['count'] / len(df.index) * 100
    df2 = df2.sort_values(by = 'percent', ascending = False)
    activities = df2.index
    df2['activity'] = activities
    df2['index'] = range(len(df2))
    df2 = df2.set_index('index')
    return df2.head(10)

Вычисляем сколько групп из моего топа и топа дрга совпадают (с погрешностью в 5 процентов)

In [ ]:
def get_my_lieber_freund(first_p, second_p):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    count = 0
    for i in range(len(first_p)):  
        for j in range(len(second_p)):
            if (first_p.loc[i, 'activity'] == second_p.loc[j, 'activity']) &  (abs(first_p.loc[i, 'percent'] - second_p.loc[j, 'percent']) < 5):
                count += 1
    return count

Мой список популярных тематик

In [6]:
my_activities = get_groups_activity(ID)
my_percents = get_percents(my_activities)
my_percents

,count,percent,activity
index,,,
0,12,10.344828,Образование
1,11,9.482759,Интернет-СМИ
2,9,7.758621,Музыкант
3,6,5.172414,Творчество
4,5,4.310345,Музыкальная группа
5,4,3.448276,Музыка
6,4,3.448276,Юмор
7,4,3.448276,Наука
8,3,2.586207,Искусство и развлечения


Получение списка друзей

In [8]:
friends = vk_api.friends.get(user_id=ID, extended=1, fields='nickname')['items']
df = pd.DataFrame(columns=['friend_name', 'count'])
i = 0
deq.appendleft(time.time())
if len(deq) == 4:
    time.sleep(max(1 + deq[3] - deq[0], 0))

Проходим по каждому другу. И вычисляем сколько у каждого друга общих типов групп с нашими. Выводим топ друзей.

In [12]:
for friend in friends: 
    try:
        fr_activities = get_groups_activity(friend['id'])
        fr_percents = get_percents(fr_activities)
        friend_name = friend['first_name'] + ' ' + friend['last_name']
        count = get_my_lieber_freund(my_percents, fr_percents)
        df.loc[i] = [friend_name, count]
        i += 1
    except Exception:
        continue

df = df.sort_values(by='count', ascending=False)

In [13]:
df

,friend_name,count
249,Максим Ратнер,6
674,Карим Гимадеев,6
618,Тимур Хайрутдинов,6
630,Аня Изгарова,6
283,Ильфат Гизатуллин,5
...,...,...
312,Евгений Разинков,0
639,Дмитрий Леонтьев,0
636,Динна Кузнецова,0
627,Вильям Нюьсом,0


In [14]:
df.head(10)

,friend_name,count
249,Максим Ратнер,6
674,Карим Гимадеев,6
618,Тимур Хайрутдинов,6
630,Аня Изгарова,6
283,Ильфат Гизатуллин,5
483,Дина Гайнуллина,5
316,Айдар Гайнутдинов-Rammsteinovich,5
43,Ярослав Петров,5
45,Владимир Сухарев,5
489,Богдан Старовойтов,5
